# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/CityWeather.csv').drop(columns = 'Unnamed: 0')
cities_df.head()

,City,Country,Lat,Lng,Date,Cloudiness,Max. Temp.,Humidity,Wind Speed
0,Thompson,CA,55.7435,-97.8558,1632201993,40,33.96,100,0.00
1,Dikson,RU,73.5069,80.5464,1632202050,75,34.86,77,9.53
2,Ushuaia,AR,-54.8000,-68.3000,1632201883,75,47.86,71,4.61
3,Kumluca,TR,36.3703,30.2869,1632202051,0,81.59,35,3.85
4,Letterkenny,IE,54.9500,-7.7333,1632202052,100,51.17,98,4.41


In [3]:
cities_df['City'].nunique()

565

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [5]:
fig = gmaps.figure(center = (45.0, -2.0), zoom_level = 1.8)
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]



heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities to fit weather conditions
sort_cities_df = cities_df.loc[(cities_df['Max. Temp.'] <= 80) & (cities_df['Max. Temp.'] >= 70)\
                                                               & (cities_df['Wind Speed'] < 10) \
                                                               & (cities_df['Cloudiness'] == 70)].reset_index()
# Drop any rows will null values.
cities_df = sort_cities_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = cities_df
hotel_df['Hotel Name'] = ''
hotel_df.head()

,index,City,Country,Lat,Lng,Date,Cloudiness,Max. Temp.,Humidity,Wind Speed,Hotel Name


In [ ]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# Set conditions for hotel searching
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))